In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

RAND_STATE = 32 # for reproducible shuffling
TT_RATIO = 0.25 # test/train

from sklearn.linear_model import Lasso,Ridge,ElasticNet, LinearRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_paris = pd.read_csv('data_paris.csv')

In [3]:
#data_paris_geo = data_paris[['lng', 'lat']]

In [4]:
data_paris.corr()

,Unnamed: 0,realSum,person_capacity,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index_norm,rest_index_norm,lng,lat,room_shared_True,room_private_True,host_is_superhost_True,weekend_yes
Unnamed: 0,1.000000,-0.164703,-0.045212,0.023824,-0.208457,-0.004463,0.050962,0.043074,0.624514,0.200687,-0.640693,-0.730778,0.029731,0.075366,0.029037,0.027435,-0.027396,0.109284
realSum,-0.164703,1.000000,0.395620,0.018087,0.104630,0.068471,0.058530,0.353561,-0.099438,-0.025620,0.252671,0.205716,-0.140388,-0.036204,-0.086654,-0.153378,0.022569,-0.017729
person_capacity,-0.045212,0.395620,1.000000,-0.010394,0.107246,-0.046916,-0.016014,0.529153,-0.062163,-0.019463,0.056676,0.058032,0.007858,0.006539,0.063102,-0.295026,-0.070217,0.004913
multi,0.023824,0.018087,-0.010394,1.000000,-0.302755,0.007057,-0.011639,0.053945,0.039875,0.027128,0.010550,-0.015428,-0.047283,-0.041856,-0.023423,0.213285,0.043066,-0.012288
biz,-0.208457,0.104630,0.107246,-0.302755,1.000000,-0.040065,-0.195606,-0.067160,-0.139010,-0.102780,0.168461,0.206530,-0.009718,0.051629,0.114735,0.081609,-0.112166,-0.026867
cleanliness_rating,-0.004463,0.068471,-0.046916,0.007057,-0.040065,1.000000,0.655019,0.043241,0.004036,-0.020198,0.016315,0.011902,-0.016337,0.002354,-0.027098,0.086487,0.214551,0.009257
guest_satisfaction_overall,0.050962,0.058530,-0.016014,-0.011639,-0.195606,0.655019,1.000000,0.095243,0.041161,0.014135,-0.032589,-0.042689,0.003679,0.029829,0.001076,0.016672,0.207250,0.019795
bedrooms,0.043074,0.353561,0.529153,0.053945,-0.067160,0.043241,0.095243,1.000000,0.009014,0.009823,-0.029479,-0.022054,0.013766,0.002824,0.005057,0.045214,0.034539,0.005514
dist,0.624514,-0.099438,-0.062163,0.039875,-0.139010,0.004036,0.041161,0.009014,1.000000,0.281302,-0.519953,-0.720074,-0.519921,0.084956,0.020926,0.064308,0.025191,-0.009628
metro_dist,0.200687,-0.025620,-0.019463,0.027128,-0.102780,-0.020198,0.014135,0.009823,0.281302,1.000000,-0.153206,-0.251922,-0.143563,-0.097493,-0.021448,-0.003536,-0.003465,-0.005199


In [5]:
data_paris = data_paris.drop(['Unnamed: 0'], axis=1)

In [6]:
from scipy import stats

# Assuming df is the name of your dataframe
z_scores = np.abs(stats.zscore(data_paris))

# Remove rows where any of the columns have a z-score greater than 3
data_paris_cleaned = data_paris[(z_scores < 3).all(axis=1)]

In [7]:
x = data_paris_cleaned.drop('realSum', axis=1)
y = data_paris_cleaned.realSum

In [8]:
# test-train split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=TT_RATIO, random_state=RAND_STATE)
x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)
x_train.describe()

,person_capacity,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index_norm,rest_index_norm,lng,lat,room_shared_True,room_private_True,host_is_superhost_True,weekend_yes
count,4554.000000,4554.000000,4554.000000,4554.000000,4554.000000,4554.000000,4554.000000,4554.000000,4554.000000,4554.000000,4554.000000,4554.000000,4554.0,4554.000000,4554.000000,4554.000000
mean,2.874616,0.210365,0.239130,9.349363,92.867809,0.919192,2.976353,0.221076,17.941175,42.552730,2.343601,48.864551,0.0,0.233202,0.146245,0.532060
std,1.143686,0.407612,0.426599,0.778184,6.643315,0.554965,1.438133,0.109390,6.858243,15.203096,0.033310,0.017351,0.0,0.422916,0.353391,0.499026
min,2.000000,0.000000,0.000000,7.000000,67.000000,0.000000,0.070000,0.000000,5.654976,11.933258,2.256630,48.819940,0.0,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,9.000000,90.000000,1.000000,1.830000,0.140000,12.820440,30.360625,2.323272,48.852320,0.0,0.000000,0.000000,0.000000
50%,2.000000,0.000000,0.000000,10.000000,94.000000,1.000000,3.000000,0.210000,16.459762,40.559221,2.347000,48.866610,0.0,0.000000,0.000000,1.000000
75%,4.000000,0.000000,0.000000,10.000000,98.000000,1.000000,3.987500,0.280000,21.916534,53.012007,2.367480,48.878195,0.0,0.000000,0.000000,1.000000
max,6.000000,1.000000,1.000000,10.000000,100.000000,2.000000,7.330000,0.590000,41.217593,89.419780,2.438590,48.901060,0.0,1.000000,1.000000,1.000000


In [9]:
x_train_const = sm.add_constant(x_train) # adding a constant

model = sm.OLS(y_train, x_train_const).fit()
predictions_train = model.predict(x_train_const)

x_test_const = sm.add_constant(x_test) # adding a constant
predictions_test = model.predict(x_test_const)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                realSum   R-squared:                       0.503
Model:                            OLS   Adj. R-squared:                  0.501
Method:                 Least Squares   F-statistic:                     306.1
Date:                Wed, 08 Mar 2023   Prob (F-statistic):               0.00
Time:                        10:44:13   Log-Likelihood:                -28845.
No. Observations:                4554   AIC:                         5.772e+04
Df Residuals:                    4538   BIC:                         5.783e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

In [10]:
sts = StandardScaler()

x_train_standarized = sts.fit_transform(x_train)
x_test_standarized = sts.transform(x_test)

x_train_standarized = pd.DataFrame(x_train_standarized)

In [11]:
x_train_const = sm.add_constant(x_train_standarized.to_numpy()) # adding a constant

model = sm.OLS(y_train, x_train_const).fit()
predictions_train = model.predict(x_train_const)

x_test_const = sm.add_constant(x_test_standarized) # adding a constant
predictions_test = model.predict(x_test_const)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                realSum   R-squared:                       0.503
Model:                            OLS   Adj. R-squared:                  0.501
Method:                 Least Squares   F-statistic:                     306.1
Date:                Wed, 08 Mar 2023   Prob (F-statistic):               0.00
Time:                        10:44:13   Log-Likelihood:                -28845.
No. Observations:                4554   AIC:                         5.772e+04
Df Residuals:                    4538   BIC:                         5.783e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        368.9552      2.024    182.312      0.0

In [12]:
lm = LinearRegression()
model = lm.fit(x_train,y_train)
model.coef_
model.intercept_

24847.499599865256

In [13]:
model.score(x_train, y_train)

0.5029234756382273

In [14]:
y_pred = pd.DataFrame(model.predict(x_test),columns = ['realSum'] )      # model prediction
y_pred_train =  pd.DataFrame(model.predict(x_train),columns = ['realSum'])

In [15]:
mse(y_test,y_pred)

19788.960678723568

In [16]:
rmse = np.sqrt(mse(y_test,y_pred))
rmse

140.6732408055049

In [17]:
##prediction on the train set
mse(y_train,y_pred_train)

18585.86803009664

In [18]:
mae(y_test,y_pred)

97.11952050308673

In [19]:
R2=r2_score(y_test,y_pred)
R2

0.5154825871856554

In [20]:
R2_test=model.score(x_test,y_test)
R2_train=model.score(x_train,y_train)
Adj_R2= 1 - (1-R2)*(len(y_test)-1)/(len(y_test)-x_test.shape[1]-1)
Adj_R2

0.5103178446106857

In [21]:
model_lasso=Lasso(alpha=0.05)

model_lasso.fit(x_train, y_train)
print(f"{model_lasso.__class__.__name__}: Train -> {model_lasso.score(x_train, y_train)}, Test -> {model_lasso.score(x_test, y_test)}")

Lasso: Train -> 0.5026035702847578, Test -> 0.5146308090279479


In [22]:
model_ridge=Ridge(alpha=1000)
model_ridge.fit(x_train, y_train)
print(f"{model_ridge.__class__.__name__}: Train -> {model_ridge.score(x_train, y_train)}, Test -> {model_ridge.score(x_test, y_test)}")

Ridge: Train -> 0.467750033431231, Test -> 0.47384331717868555


In [23]:
model = DecisionTreeRegressor()
model.fit(x_train, y_train)

print("X_train has %d rows and %d columns"  %(x_train.shape[0],x_train.shape[1]))
print("-----------------------------------")
print("The coefficient of determination for the test data is R2=%.2f"
      %(model.score(x_test, y_test)))
print("The coefficient of determination for the train data is R2=%.2f"
      %(model.score(x_train, y_train)))


X_train has 4554 rows and 16 columns
-----------------------------------
The coefficient of determination for the test data is R2=0.58
The coefficient of determination for the train data is R2=1.00


In [24]:
scores=cross_val_score(model, x_train, y_train, cv=5)
print("Cross validation scores: ", scores)
print("Score stats: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Cross validation scores:  [0.36609373 0.5340081  0.37124228 0.41789347 0.4457489 ]
Score stats: 0.43 accuracy with a standard deviation of 0.06


In [25]:
model1 = DecisionTreeRegressor()
model2 = LinearRegression()
model3 = KNeighborsRegressor()

model_pipeline = [model1, model2, model3]
model_names = ['Regression Tree', 'Linear Regression', 'KNN']
scores = {}
i=0
for model in model_pipeline:
    mean_score = np.mean(cross_val_score(model, x_train, y_train, cv=5))
    scores[model_names[i]] = mean_score
    i = i+1
print(scores)

{'Regression Tree': 0.43267750250389564, 'Linear Regression': 0.4969402629941043, 'KNN': 0.2957333874438728}


In [26]:
data_paris_cleaned.to_excel('data_paris.xlsx')

In [27]:
#data_paris_geo.to_excel('data_paris_geo.xlsx')

In [28]:
#paris = pd.concat((data_paris_cleaned, data_paris_geo), axis=1)
#paris.to_excel('paris.xlsx')

In [29]:
import pysal

# Create spatial weights matrix
w = pysal.lib.weights.Queen.from_dataframe(data_paris)

# Create spatial lag variable
y_lag = pysal.lib.weights.lag_spatial(w, data['y'])

# Ordinary least squares (OLS) regression
model = pysal.model.spreg.OLS(y, x, w=w, name_y='y', name_x=['x1', 'x2'])

# Get summary of regression results
print(model.summary)

AttributeError: partially initialized module 'fiona' has no attribute '_loading' (most likely due to a circular import)

In [ ]:
from shapely.geometry import Point
import geopandas as gpd
from pysal.model import spreg
from pysal.lib import weights

In [ ]:

from scipy import stats
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import osmnx as ox


In [ ]:
query = {'city' : 'Paris'}

boundaries = ox.geocode_to_gdf(query, buffer_dist=1000)